# Isi Penting Generator: Article Style

Generate a long text output with the style of an article when given important facts (isi penting in Malay).

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/isi-penting-generator-article-style](https://github.com/huseinzol05/Malaya/tree/master/example/isi-penting-generator-article-style).
    
</div>

<div class="alert alert-warning">

The results you see here are generated using stochastic methods. Learn more about the stochastic process on <a href="https://en.wikipedia.org/wiki/Stochastic_process" target="_blank">Wikipedia</a>
    
</div>

In [1]:
%%time
import malaya
from pprint import pprint

CPU times: user 3.35 s, sys: 302 ms, total: 3.65 s
Wall time: 3.6 s


/home/maguswyvern/PythonVenvs/dev-malaya/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/maguswyvern/PythonVenvs/dev-malaya/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### List all available HuggingFace transformers

The `malaya` library has a built in function to find all available transformers for this task. As of writing we have two transformers which are:

1. mesolitica/finetune-isi-penting-generator-t5-base-standard-bahasa-cased <br>
https://huggingface.co/mesolitica/finetune-isi-penting-generator-t5-base-standard-bahasa-cased
   
2. mesolitica/finetune-isi-penting-generator-t5-small-standard-bahasa-cased <br>
https://huggingface.co/mesolitica/finetune-isi-penting-generator-t5-small-standard-bahasa-cased

In [2]:
malaya.generator.isi_penting.available_huggingface

{'mesolitica/finetune-isi-penting-generator-t5-small-standard-bahasa-cased': {'Size (MB)': 242,
  'ROUGE-1': 0.24620333,
  'ROUGE-2': 0.05896076,
  'ROUGE-L': 0.15158954,
  'Suggested length': 1024},
 'mesolitica/finetune-isi-penting-generator-t5-base-standard-bahasa-cased': {'Size (MB)': 892,
  'ROUGE-1': 0.24620333,
  'ROUGE-2': 0.05896076,
  'ROUGE-L': 0.15158954,
  'Suggested length': 1024}}

---

### Load HuggingFace

The Generator transformer in `malaya` is quite unique, most of the text generative model we found on the internet like GPT2 or Markov simply just continue the prefix input from user, but not for our Generator transformer. 

We want to generate an article or karangan like high school when the users give 'isi penting' or important facts for the article.

```python
def huggingface(
    model: str = 'mesolitica/finetune-isi-penting-generator-t5-base-standard-bahasa-cased',
    force_check: bool = True,
    **kwargs,
):
    """
    Load HuggingFace model to generate text based on isi penting.

    Parameters
    ----------
    model: str, optional (default='mesolitica/finetune-isi-penting-generator-t5-base-standard-bahasa-cased')
        Check available models at `malaya.generator.isi_penting.available_huggingface`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.IsiPentingGenerator
    """
```

In [3]:
model = malaya.generator.isi_penting.huggingface()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Here is the `generate` function and the parameters it expects. 

```python
def generate(
    self,
    strings: List[str],
    mode: str = 'surat-khabar',
    **kwargs,
):
    """
    generate a long text given a isi penting.

    Parameters
    ----------
    strings : List[str]
    mode: str, optional (default='surat-khabar')
        Mode supported. Allowed values:

        * ``'surat-khabar'`` - news style writing.
        * ``'tajuk-surat-khabar'`` - headline news style writing.
        * ``'artikel'`` - article style writing.
        * ``'penerangan-produk'`` - product description style writing.
        * ``'karangan'`` - karangan sekolah style writing.

    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

---

### Benefits of HuggingFace

With the `generate` method you can use Greedy, Beam, Sampling, Nucleus decoder and so much more, read more about it on the [HuggingFace Article on How to Generate](https://huggingface.co/blog/how-to-generate). And recently, HuggingFace also released a new article [Introducing Csearch](https://huggingface.co/blog/introducing-csearch)

Let's give a few lines of important facts or isi penting for the model to use to generate text.

In [4]:
isi_penting = ['Neelofa tetap dengan keputusan untuk berkahwin akhir tahun ini',
              'Long Tiger sanggup membantu Neelofa',
              'Tiba-tiba Long Tiger bergaduh dengan Husein Zolkepli']

As above, we can give any isi penting even if it does not make any sense. Now we'll use the `generate` method and pass in a few of the vector arguments mentioned in a previous linked article by HuggingFace.

In [5]:
pprint(model.generate(isi_penting, mode = 'artikel',
    do_sample=True,
    max_length=256,
    top_k=50, 
    top_p=0.95))

['Dia berkahwin dengan Husein Zolkepli pada tahun 2003. Dengan hubungan ini, '
 'Zolkepli dan Neelofa berkahwin untuk kali kedua. Ketika Husein Zolkepli '
 'bergaduh dengan Neelofa, kedua-duanya bergaduh. Ini adalah perkahwinan kedua '
 'mereka. Pasangan itu berkahwin dalam perkahwinan yang sama.']


At this point if you get the following error:

<div class="alert alert-warning">
    
FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml.

</div>

Try installing `lxml` with `pip install lxml`

---

Previously we set the `top_k` parameter to `50`. A higher `top_k` value means the model considers more candidates, potentially leading to more diversity in the generated text but also increasing the computational cost.

Now let's try lowering the parameter down and introduce the `penalty_alpha` argument to decrease randomness.

In [6]:
pprint(model.generate(isi_penting, mode = 'artikel',
    do_sample=True, 
    max_length=256, 
    penalty_alpha=0.8, top_k=4))

['Neelofa adalah seorang gadis yang sangat baik, dan dia tidak mempunyai masa '
 'untuk berkahwin. Dia mempunyai seorang anak perempuan bernama Husein '
 'Zolkepli dan seorang anak lelaki bernama Neelofa. Neelofa dan Husein '
 'Zolkepli mempunyai seorang anak perempuan bernama Neelofa.']


Try changing the `isi_penting` variable and rerun the generate method again for more results.

In [7]:
isi_penting = ['Astronomi (dari) adalah sains semula jadi yang mengkaji objek dan fenomena cakerawala.',
 'Ia menggunakan matematik, fizik, dan kimia untuk menjelaskan asal usul dan evolusi mereka.',
 'Objek yang menarik termasuk planet, bulan, bintang, nebula, galaksi, dan komet.',
 'Fenomena yang relevan termasuk letupan supernova, ledakan sinar gamma, kusar, blazar, pulsar, dan radiasi latar belakang gelombang mikro kosmik.',
 'Secara umum, astronomi mengkaji semua yang berasal dari luar atmosfer Bumi.',
 'Kosmologi adalah cabang astronomi.',
 'Ia mengkaji Alam Semesta secara keseluruhan.']

In [8]:
pprint(model.generate(isi_penting, mode = 'artikel',
    do_sample=True,
    max_length=256,
    top_k=50, 
    top_p=0.95))

['Istilah "astrofizik" digunakan pada abad ke-19, dengan pengembangan istilah '
 '"astrofizik" pada tahun 1836, kemudian untuk menggambarkan bidang di mana '
 'pemerhatian telah dihasilkan, dengan cara yang sama sekali tidak dikenali '
 'dalam sastera, dalam pelbagai disiplin. Astrofizik moden merangkumi banyak '
 'disiplin; dari astronomi sederhana hingga matematik, fizik, dan kimia (dalam '
 'disiplin saintifik hingga matematik, fizik, kimia), dan fizik (dalam bidang '
 'fizik, fizik, dan kimia), semuanya menyumbang kepada bidang ini. Ahli '
 'astronomi secara umum menganggap alam semesta sebagai alam semesta fizikal, '
 'bukan alam semesta semula jadi. Terdapat "teori" saintifik untuk menyiasat '
 'objek, dan, tidak semestinya, adalah bahawa setiap objek akan tetap '
 'berhubungan dengan objek dan alam semesta. Astronomi adalah sains sains. '
 'Banyak ahli astronomi menggunakan sains untuk mengkaji objek dan alam '
 'semesta. Ini bukan sains, tetapi sebaliknya ahli astronomi. Bidang

In [9]:
pprint(model.generate(isi_penting, mode = 'artikel',
    do_sample=True, 
    max_length=256, 
    penalty_alpha=0.8, top_k=4))

['Dalam astronomi, "astronomi" adalah sains semula jadi. Ahli astronomi sering '
 'mengkaji objek dan fenomena cakerawala, termasuk bintang. Astronomi mengkaji '
 'semua perkara yang berasal dari luar atmosfer Bumi. Astronomi juga mengkaji '
 '"kecerdasan alam semesta" dan "kecerdasan Bumi". Astronomi adalah sains '
 'semula jadi, yang mengkaji alam semesta secara keseluruhan. Ia mengkaji '
 'semua perkara yang berasal dari luar atmosfer. Ahli astronomi mengkaji semua '
 'perkara yang berasal dari luar atmosfer Bumi dan mengintegrasikannya dengan '
 'fizik dan fizik. Astronomies adalah sains yang berkaitan dengan sains dan '
 'sejarah, termasuk sains semula jadi, sains Bumi. Astronomi merangkumi sains '
 'yang berkaitan dengan sains, sejarah, dan sains Bumi, termasuk sains Bumi, '
 'fizik, dan kimia, dan sains Bumi. Astronomies mengkaji alam semesta dengan '
 'pelbagai cara, termasuk alam semesta, sains Bumi, dan sains Bumi. '
 'Astronomies juga mengkaji "kecerdasan alam semesta", "kec